### Caderno para estudo de simulações de dinâmica molecular

Um aspecto importante para as simulações moleculares como monte-carlo e dinâmica molecular diz respeito à forma em que as partículas  interagem e, também, como tais interações podem ser representativas para sistemas reais. O primeiro aspecto relevante diz respeito às condições periódicas de contorno (PBC, do ingles periodic boundary conditions). 

Tais métodos computacionais visam prover informações sobre propriedades de uma amostra macroscópica. Normalmente, o número de graus de liberdade simulados em sistemas tipicamente abordados em MD e MC é da ordem de milhares até milhões de átomos. 

Em um sistema 3D com N partículas sem fronteiras, a fração de moléculas que está na superfície é proporcional N^(-1/3). Por exemplo, em um cristal cúbico com 1000 átomos , cerca de 49 % dos átomos estarão na fronteira. Assim, as propriedades de tais sistemas serão fortemente dominadas por efeitos de fronteira. Sendo, desta forma, impossível de reproduzir propriedades bulk.

Assim, para a correta repodução dos sistemas é preciso escolher condições de contorno que imitam a presença de um bulk infinito de partículas com mesma identidade das N existentes no sistema. Isto é frequentemente atingido por meio da aplicação de propriedades periódicas de contorno.


In [1]:
function minimum_image(rc, L)
    if rc < - L/2
        rc = rc + L
    elseif rc >= L/2
        rc = rc - L
    end 
    return rc    
end

minimum_image (generic function with 1 method)

Agora, podemos considerar um aspecto muito importante que diz respeito à forma com que as partículas interagem. Vamos considerar um caso de um sistema com interações de curta distância  (short-ranged) e que essas interações sejam aditivas por pares (pairwise-additive interactions). Neste contexto, as interações serem de curta distância significam que a energia potencial total de uma dada partícula $i$ será dominada por interações com partículas vizinhas que estão há uma distância menor do que $r_c$. O erro de ignorar interações entre partículas que estão em distâncias maiores do que $r_c$ torna-se menor ao passo que consideramos $r_c$ maior.

Se usamos condições periódicas de contorno:

*  se $r_c$ é maior do que $L/2$, precisamos considerar a interação da partícula $i$ apenas com a imagem periódica mais próxima de qualquer outra partícula $j$.   

In [2]:
function setlat(nx, ny, nz, vol)
    # Calculate the lattice constant (unit cell edge length)
    npart = 4 * nx * ny * nz  # 4 particles per unit cell
    a0 = (vol / (nx * ny * nz))^(1/3)
    
    # Initialize arrays for positions
    x0 = Vector{Float64}(undef, npart)
    y0 = similar(x0)
    z0 = similar(x0)
    
    # Initialize counters and center of mass
    i = 0
    xcm0 = 0.0
    ycm0 = 0.0
    zcm0 = 0.0
    
    # Generate positions
    for iz in 1:2*nz
        for iy in 1:2*ny
            for ix in 1:2*nx
                if (ix + iy + iz) % 2 == 0
                    i += 1
                    x0[i] = 0.5 * a0 * (ix - 1 + 0.5 * ((iy + iz) % 2))
                    y0[i] = 0.5 * a0 * (iy - 1 + 0.5 * ((ix + iz) % 2))
                    z0[i] = 0.5 * a0 * (iz - 1 + 0.5 * ((ix + iy) % 2))
                    
                    # Accumulate for center of mass
                    xcm0 += x0[i]
                    ycm0 += y0[i]
                    zcm0 += z0[i]
                end
            end
        end
    end
    
    # Calculate center of mass
    xcm0 /= npart
    ycm0 /= npart
    zcm0 /= npart
    
    return x0, y0, z0, xcm0, ycm0, zcm0, a0
end

setlat (generic function with 1 method)

In [3]:
x, y, z, xcm, ycm, zcm, a0 = setlat(2, 2, 2, 100.0)

([1.1603972084031946, 3.4811916252095836, 0.5801986042015973, 2.9009930210079866, 1.1603972084031946, 3.4811916252095836, 0.5801986042015973, 2.9009930210079866, 0.5801986042015973, 2.9009930210079866  …  0.5801986042015973, 2.9009930210079866, 0.5801986042015973, 2.9009930210079866, 1.1603972084031946, 3.4811916252095836, 0.5801986042015973, 2.9009930210079866, 1.1603972084031946, 3.4811916252095836], [0.5801986042015973, 0.5801986042015973, 1.1603972084031946, 1.1603972084031946, 2.9009930210079866, 2.9009930210079866, 3.4811916252095836, 3.4811916252095836, 0.5801986042015973, 0.5801986042015973  …  3.4811916252095836, 3.4811916252095836, 0.5801986042015973, 0.5801986042015973, 1.1603972084031946, 1.1603972084031946, 2.9009930210079866, 2.9009930210079866, 3.4811916252095836, 3.4811916252095836], [0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 0.5801986042015973, 1.1603972084031946, 1.160397

In [4]:
using GLMakie  # Using GLMakie for 3D interactive plots

# FCC lattice generation function (same as before)
function setlat(nx, ny, nz, vol)
    npart = 4 * nx * ny * nz
    a0 = (vol / (nx * ny * nz))^(1/3)
    
    x0 = Vector{Float64}(undef, npart)
    y0 = similar(x0)
    z0 = similar(x0)
    
    i = 0
    xcm0 = 0.0
    ycm0 = 0.0
    zcm0 = 0.0
    
    for iz in 1:2*nz
        for iy in 1:2*ny
            for ix in 1:2*nx
                if (ix + iy + iz) % 2 == 0
                    i += 1
                    x0[i] = 0.5 * a0 * (ix - 1 + 0.5 * ((iy + iz) % 2))
                    y0[i] = 0.5 * a0 * (iy - 1 + 0.5 * ((ix + iz) % 2))
                    z0[i] = 0.5 * a0 * (iz - 1 + 0.5 * ((ix + iy) % 2))
                    
                    xcm0 += x0[i]
                    ycm0 += y0[i]
                    zcm0 += z0[i]
                end
            end
        end
    end
    
    xcm0 /= npart
    ycm0 /= npart
    zcm0 /= npart
    
    return x0, y0, z0, xcm0, ycm0, zcm0, a0
end

# Generate the lattice (2x2x2 unit cells, volume = 100)
nx, ny, nz = 2, 2, 2
vol = 100.0
x, y, z, xcm, ycm, zcm, a0 = setlat(nx, ny, nz, vol)

# Create the plot
fig = Figure(resolution = (800, 600))
ax = Axis3(fig[1, 1], title = "FCC Lattice ($nx×$ny×$nz unit cells)",
           xlabel = "X", ylabel = "Y", zlabel = "Z")

# Plot the atoms as spheres
meshscatter!(ax, x, y, z, markersize = 0.15*a0, color = :lightblue)

# (Optional) Draw unit cell edges for visualization
unit_cell_lines = [
    # Bottom face
    [0, 0, 0] => [a0, 0, 0],
    [a0, 0, 0] => [a0, a0, 0],
    [a0, a0, 0] => [0, a0, 0],
    [0, a0, 0] => [0, 0, 0],
    # Top face
    [0, 0, a0] => [a0, 0, a0],
    [a0, 0, a0] => [a0, a0, a0],
    [a0, a0, a0] => [0, a0, a0],
    [0, a0, a0] => [0, 0, a0],
    # Vertical edges
    [0, 0, 0] => [0, 0, a0],
    [a0, 0, 0] => [a0, 0, a0],
    [0, a0, 0] => [0, a0, a0],
    [a0, a0, 0] => [a0, a0, a0]
]

for (start_pt, end_pt) in unit_cell_lines
    lines!(ax, [start_pt[1], end_pt[1]], [start_pt[2], end_pt[2]], [start_pt[3], end_pt[3]],
           color = :black, linewidth = 1.5)
end

# Center the view on the lattice
limits!(ax, -0.1*a0, (2*nx)*0.5*a0 + 0.1*a0, 
        -0.1*a0, (2*ny)*0.5*a0 + 0.1*a0, 
        -0.1*a0, (2*nz)*0.5*a0 + 0.1*a0)

fig

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\vinic\.julia\packages\Makie\6KcTF\src\scenes.jl:238


In [5]:
using Random

function initv(temp, x0, y0, z0, npart, dt)
    # Initialize arrays
    vx = Vector{Float64}(undef, npart)
    vy = similar(vx)
    vz = similar(vx)
    xm = similar(vx)
    ym = similar(vx)
    zm = similar(vx)
    
    sumv = zeros(3)  # sumv[1] = sum(vx), sumv[2] = sum(vy), sumv[3] = sum(vz)
    sumv2 = 0.0
    
    # Generate velocities from Maxwell-Boltzmann distribution
    for i in 1:npart
        # Generate Gaussian velocities using Box-Muller transform
        r1, r2 = rand(), rand()
        vx[i] = sqrt(-2 * log(r1)) * cos(2π * r2)
        r1, r2 = rand(), rand()
        vy[i] = sqrt(-2 * log(r1)) * cos(2π * r2)
        r1, r2 = rand(), rand()
        vz[i] = sqrt(-2 * log(r1)) * cos(2π * r2)
        
        sumv .+= (vx[i], vy[i], vz[i])
    end
    
    # Remove center-of-mass velocity
    sumv ./= npart
    for i in 1:npart
        vx[i] -= sumv[1]
        vy[i] -= sumv[2]
        vz[i] -= sumv[3]
        sumv2 += vx[i]^2 + vy[i]^2 + vz[i]^2
    end
    
    # Scale velocities to desired temperature
    nf = 3 * npart - 3  # Degrees of freedom (subtract 3 for COM motion)
    fs = sqrt(temp / (sumv2 / nf))
    
    vx .*= fs
    vy .*= fs
    vz .*= fs
    
    # Set previous positions for Verlet integration
    for i in 1:npart
        xm[i] = x0[i] - vx[i] * dt
        ym[i] = y0[i] - vy[i] * dt
        zm[i] = z0[i] - vz[i] * dt
    end
    
    return vx, vy, vz, xm, ym, zm
end

initv (generic function with 1 method)

In [ ]:
function md()

    setlat() # function to initialize positions
    initv(temp)
    t = 0
    while t < tmax
        FandE # computes forces and total energy
        Integrate-V # integrate equations of motion
        t = t + delt
        sample # function to sample avarages


end

In [7]:
# Force and energy calculation
function FandE(x, y, z, box, rc)
    npart = length(x)
    fx = zeros(npart)
    fy = zeros(npart)
    fz = zeros(npart)
    en = 0.0
    rc2 = rc^2
    
    for i in 1:npart-1
        for j in i+1:npart
            # Periodic boundary conditions
            xr = x[i] - x[j]
            xr -= box * round(xr/box)
            
            yr = y[i] - y[j]
            yr -= box * round(yr/box)
            
            zr = z[i] - z[j]
            zr -= box * round(zr/box)
            
            r2 = xr^2 + yr^2 + zr^2
            
            if r2 < rc2
                r2i = 1/r2
                r2im1 = r2i - 1.0
                rc2r2im1 = rc2*r2i - 1.0
                
                # Energy contribution
                en += r2im1 * rc2r2im1^2
                
                # Force calculation
                ff = 6.0 * r2i^2 * rc2r2im1 * (rc2r2im1 - 2.0)
                
                fx[i] += ff * xr
                fy[i] += ff * yr
                fz[i] += ff * zr
                
                fx[j] -= ff * xr
                fy[j] -= ff * yr
                fz[j] -= ff * zr
            end
        end
    end
    
    return fx, fy, fz, en
end

FandE (generic function with 1 method)

In [8]:
function integrate_v!(x, xm, fx, delt, nf, en)
    npart = length(x)
    sumv = 0.0
    sumv2 = 0.0
    
    # Verlet integration and velocity calculation
    for i in 1:npart
        xx = 2*x[i] - xm[i] + delt^2 * fx[i]  # Verlet position update
        vi = (xx - xm[i]) / (2*delt)          # Velocity calculation
        
        sumv += vi
        sumv2 += vi^2
        
        # Update positions
        xm[i] = x[i]
        x[i] = xx
    end
    
    # Calculate temperature and total energy
    temp = sumv2 / nf
    etot = (en + 0.5*sumv2) / npart
    
    return temp, etot, sumv/npart  # Returns temperature, energy per particle, and COM velocity
end

integrate_v! (generic function with 1 method)

In [ ]:
function vel_verlet!(x, v, f, delt, m, force_calculator)
    """
    Velocity Verlet integrator
    Args:
        x: positions (will be updated)
        v: velocities (will be updated)
        f: forces from previous step
        delt: time step
        m: particle mass (can be scalar or vector)
        force_calculator: function that takes positions and returns (fx, fy, fz, en)
    Returns:
        K: kinetic energy
        en: potential energy
        com_v: center-of-mass velocity
    """
    npart = length(x)
    delt2 = delt/2
    K = 0.0
    sumv = zeros(3)  # For COM velocity calculation
    
    # First half-step velocity update and position update
    @inbounds for i in 1:npart
        v[i] += f[i] * delt2 / m
        x[i] += v[i] * delt
    end
    
    # Get new forces
    f_new, en = force_calculator(x)
    
    # Second half-step velocity update and kinetic energy calculation
    @inbounds for i in 1:npart
        v[i] += f_new[i] * delt2 / m
        vi_sq = v[i]^2
        K += 0.5 * m * vi_sq
        sumv .+= (v[i], 0.0, 0.0)  # Simplified for 1D - extend to 3D
    end
    
    # Update forces for next step
    copyto!(f, f_new)
    
    # COM velocity (per dimension)
    com_v = sumv ./ (npart * m)
    
    return K, en, com_v
end

function vel_verlet_3d!(x, y, z, vx, vy, vz, fx, fy, fz, delt, m, box, rc)
    """
    3D Velocity Verlet integrator for MD
    Args:
        Positions and velocities for x,y,z
        Forces fx, fy, fz from previous step
        delt: time step
        m: mass (scalar)
        box: simulation box size
        rc: cutoff radius
    Returns:
        K: kinetic energy
        en: potential energy
        com_v: center-of-mass velocity vector
    """
    npart = length(x)
    delt2 = delt/2
    K = 0.0
    sumv = zeros(3)
    
    # Half-step velocity update and full position update
    @inbounds for i in 1:npart
        vx[i] += fx[i] * delt2 / m
        vy[i] += fy[i] * delt2 / m
        vz[i] += fz[i] * delt2 / m
        
        x[i] += vx[i] * delt
        y[i] += vy[i] * delt
        z[i] += vz[i] * delt
    end
    
    # Get new forces
    fx_new, fy_new, fz_new, en = FandE(x, y, z, box, rc)
    
    # Second half-step velocity update and KE calculation
    @inbounds for i in 1:npart
        vx[i] += fx_new[i] * delt2 / m
        vy[i] += fy_new[i] * delt2 / m
        vz[i] += fz_new[i] * delt2 / m
        
        vi_sq = vx[i]^2 + vy[i]^2 + vz[i]^2
        K += 0.5 * m * vi_sq
        sumv .+= (vx[i], vy[i], vz[i])
    end
    
    # Update forces for next step
    copyto!(fx, fx_new)
    copyto!(fy, fy_new)
    copyto!(fz, fz_new)
    
    # COM velocity
    com_v = sumv ./ npart  # Assuming m=1
    
    return K, en, com_v
end


function compare_integrators()
    # Common setup
    nx, ny, nz = 2, 2, 2
    vol = 100.0
    temp = 1.0
    dt = 0.001
    rc = 2.0
    nsteps = 1000
    m = 1.0  # Using unit mass
    
    # Initialize system
    x, y, z, box, npart = setlat(nx, ny, nz, vol)
    vx, vy, vz, xm, ym, zm = initv(temp, x, y, z, npart, dt)
    nf = 3npart - 3
    
    # For position Verlet
    fx_p, fy_p, fz_p, en_p = FandE(x, y, z, box, rc)
    
    # For velocity Verlet (need to make copies)
    x_v = copy(x); y_v = copy(y); z_v = copy(z)
    vx_v = copy(vx); vy_v = copy(vy); vz_v = copy(vz)
    fx_v, fy_v, fz_v, en_v = FandE(x_v, y_v, z_v, box, rc)
    
    for step in 1:nsteps
        # Position Verlet
        temp_p, etot_p, com_v_p = integrate_v!(x, xm, fx_p, dt, nf, en_p)
        fx_p, fy_p, fz_p, en_p = FandE(x, y, z, box, rc)
        
        # Velocity Verlet
        K_v, en_v, com_v_v = vel_verlet_3d!(
            x_v, y_v, z_v, vx_v, vy_v, vz_v, 
            fx_v, fy_v, fz_v, dt, m, box, rc
        )
        temp_v = 2K_v / nf
        
        if step % 100 == 0
            println("Step $step:")
            println("  Position Verlet: T=$(temp_p) E=$etot_p COM_v=$(norm(com_v_p))")
            println("  Velocity Verlet: T=$(temp_v) E=$(en_v + K_v)/$npart COM_v=$(norm(com_v_v))")
        end
    end
end

In [6]:
nx, ny, nz = 2, 2, 2
vol = 100.0
x0, y0, z0, xcm, ycm, zcm, a0 = setlat(nx, ny, nz, vol)
npart = length(x0)

temp = 1.0  # Desired initial temperature
dt = 0.001  # Time step for MD simulation
vx, vy, vz, xm, ym, zm = initv(temp, x0, y0, z0, npart, dt)

([1.4732029886675657, 1.0051041243323935, 0.3133539529957237, -1.712375913100605, 0.7219224735903631, 0.7048370421441061, 0.05647542229656849, -1.6553195469907462, 1.1787164571871602, -0.36763480578988056  …  -1.2929010835015629, 0.27363419852306176, 0.6229699172265207, -2.21100045145314, -0.5848146366134517, -0.5018874697172895, -0.3875398511283051, 1.4661738364182129, 0.12677937937660957, -1.08802124180418], [-0.6759751299658173, -2.2919012813706225, 1.722027874070089, -0.31043207819365054, -1.5803883104218428, -0.11632496561873572, 1.0152805645360512, -0.5157157987690401, -0.6674696372972477, 1.1505087699368888  …  0.6888912837530523, 2.204892256374465, -1.12726646710357, 0.1813589844758733, 0.3022482889321866, 0.06425878624171125, 0.8028133465418767, -1.5424465182356784, -0.15570081434047944, 0.97468830235141], [0.8437867527277519, -1.717258427094374, 0.9496009709391136, -0.8930458933584102, -0.7809663949223696, 0.6809888376235091, 2.0208481276447436, 1.8005147237282537, -0.0675538